In [303]:
import pandas as pd
import requests
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [304]:
#Import dataframes
rep_df =pd.read_csv('rep_merge_07102019.csv',thousands=',')
dem_df = pd.read_csv('dem_merge_07102019.csv',thousands=',')

In [305]:
#add a tag so we can tell these two apart
rep_df['tag']='rep'
dem_df['tag']='dem'

In [306]:
rep_df.head(3)

,Unnamed: 0,funds,link,location,summary,title,created_date,tag
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-a...,"Dallas, TX",SMU College Republicans is bringing comedian S...,Steven Crowder Live at SMU!,"Created March 7, 2018",rep
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republica...,"Brookline, MA",NOTE: We reached our goal and then some in les...,Dems help reopen a NC Repub office,"Created October 16, 2016",rep
2,2,"\n$13,021 raised of $20,000",https://www.gofundme.com/troopersanchez,"Mcallen, TX","￼ Moises Sanchez, a Texas State Trooper, was s...",Funds for Trooper Moises Sanchez,NaN,rep


In [307]:
df=rep_df.append(dem_df)

In [308]:
df=df.drop_duplicates(subset='link')

In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1716 entries, 0 to 999
Data columns (total 8 columns):
Unnamed: 0      1716 non-null int64
funds           1716 non-null object
link            1716 non-null object
location        1716 non-null object
summary         1716 non-null object
title           1716 non-null object
created_date    1438 non-null object
tag             1716 non-null object
dtypes: int64(1), object(7)
memory usage: 120.7+ KB


In [310]:
#it would be good to delete rows with different currencies
#splitting the funds raised into raised and goal
df=df[df['funds'].str.contains('\$')==True]
df['raised']=df.funds.str.extract(r"(.*) raised of")
df['goal']=df.funds.str.extract(r".* raised of (.*)")

In [223]:
#for some reason some dates are missing but I seem to be able to find the dates for the individual links
no_date_df=df[df['created_date'].isnull()==True]['link']

In [174]:
no_date_df

2                https://www.gofundme.com/troopersanchez
8                     https://www.gofundme.com/templecrs
33               https://www.gofundme.com/supportcsufgop
35     https://www.gofundme.com/republicans-helping-g...
40     https://www.gofundme.com/electing-republican-w...
41     https://www.gofundme.com/any-republican-but-ro...
49     https://www.gofundme.com/muslim-politician-lea...
54                       https://www.gofundme.com/w5x8ws
65     https://www.gofundme.com/support-our-senate-re...
67     https://www.gofundme.com/need-help-from-fellow...
73     https://www.gofundme.com/nmn67u-republican-or-...
74     https://www.gofundme.com/are-you-a-democrat-or...
75     https://www.gofundme.com/national-teenage-repu...
76     https://www.gofundme.com/valley-pride-dominica...
79        https://www.gofundme.com/mp9ap-nursing-classes
81           https://www.gofundme.com/VoteRepublican2020
82     https://www.gofundme.com/republicans-put-your-...
89     https://www.gofundme.com

In [120]:
no_dates=[]

In [122]:
#trying to find the dates again
for link in no_date_df:
    dt= {}
    response = requests.get(link)
    doc = BeautifulSoup(response.text)
    date=doc.find(class_='created-date')
    print(link)
    try:
        print(date.text)
        dt['link']=link
        dt['created_date']=date.text
        no_dates.append(dt)
    except:
        print(date)

https://www.gofundme.com/troopersanchez
None
https://www.gofundme.com/templecrs
Created June 4, 2019
https://www.gofundme.com/supportcsufgop
Created January 29, 2019
https://www.gofundme.com/republicans-helping-gov039t-shutdown-employees
None
https://www.gofundme.com/electing-republican-women-training
Created February 21, 2019
https://www.gofundme.com/any-republican-but-rob-portman
None
https://www.gofundme.com/muslim-politician-leaving-the-republican-party
None
https://www.gofundme.com/w5x8ws
None
https://www.gofundme.com/support-our-senate-republicans
None
https://www.gofundme.com/need-help-from-fellow-republicans
None
https://www.gofundme.com/nmn67u-republican-or-democrat
Created June 11, 2019
https://www.gofundme.com/are-you-a-democrat-or-a-republican
None
https://www.gofundme.com/national-teenage-republican-conference-leadership
None
https://www.gofundme.com/valley-pride-dominican-republican-baseball
Created May 26, 2019
https://www.gofundme.com/mp9ap-nursing-classes
None
https://

https://www.gofundme.com/1ngcjizec0
None
https://www.gofundme.com/pro-life-is-the-right-life
Created May 25, 2019
https://www.gofundme.com/school-lunch-for-children-in-need
None
https://www.gofundme.com/1naqhw5e5c
Created May 23, 2019
https://www.gofundme.com/1n82lgs9pc
None
https://www.gofundme.com/helping-boomers-witness-climate-change
Created May 20, 2019
https://www.gofundme.com/3jazua-border-wall-trust-fund-act
None
https://www.gofundme.com/latetermretroactive-abortion-for-mitch-mcconnell
Created May 18, 2019
https://www.gofundme.com/help-mark039s-fight-for-justice
None
https://www.gofundme.com/maga-hats-for-kids
Created May 15, 2019
https://www.gofundme.com/profits-go-to-government-for-building-of-the-wall
None
https://www.gofundme.com/c447zb-for-the-wall
None
https://www.gofundme.com/ujsrhm-build-that-wall
None
https://www.gofundme.com/righteous-leaders-will-take-the-kingdom
None
https://www.gofundme.com/quality-of-life-billboards
Created May 4, 2019
https://www.gofundme.com/the

https://www.gofundme.com/fund-rides-for-illegal-aliens-to-sanctuary-cities
Created April 13, 2019
https://www.gofundme.com/da5nbu-save-del-mar-mesa-preserve
None
https://www.gofundme.com/uxqgh-we-the-people-stand-united
None
https://www.gofundme.com/qj45y4-green
Created March 7, 2019
https://www.gofundme.com/MadamMississippi
Created February 28, 2019
https://www.gofundme.com/SurvivingEdBuck
None
https://www.gofundme.com/g9ya6-build-the-roof
Created February 23, 2019


In [311]:
no_dates_df=pd.DataFrame(no_dates)

In [312]:
no_dates_df.head(3)

,created_date,link
0,"Created June 4, 2019",https://www.gofundme.com/templecrs
1,"Created January 29, 2019",https://www.gofundme.com/supportcsufgop
2,"Created February 21, 2019",https://www.gofundme.com/electing-republican-w...


In [313]:
df=df.merge(no_dates_df,on='link',how='left')

In [314]:
df['col_z'] = df['created_date_x'].where(df['created_date_x'].notnull(), df['created_date_y'])
df = df.drop(['created_date_x','created_date_y'],axis=1)

In [315]:
df.tail(5)

,Unnamed: 0,funds,link,location,summary,title,tag,raised,goal,col_z
1656,995,\n$0 raised of $400,https://www.gofundme.com/shut-down-child-proso...,"Seattle, WA",I need funds for room rentals for community ca...,Shut Down Child Proson Camps Meetings Seattle,dem,$0,$400,"Created February 25, 2019"
1657,996,"\n$0 raised of $3,000",https://www.gofundme.com/SurvivingEdBuck,"Brooklyn, NY",I am a Survivor Of Ed Buck. I was Visiting Cal...,Mental Destruction,dem,$0,"$3,000",NaN
1658,997,\n$0 raised of $1.0M,https://www.gofundme.com/Online-identity-theft,"Apple Valley, CA",We require financial assistance in order to su...,Fighting Childhood Online Identity Theft,dem,$0,$1.0M,"Created February 23, 2019"
1659,998,\n$0 raised of $500.0M,https://www.gofundme.com/g9ya6-build-the-roof,"Clifton Park, NY",Too many people say that a wall to protect the...,Build the Roof,dem,$0,$500.0M,"Created February 23, 2019"
1660,999,"\n$0 raised of $150,000",https://www.gofundme.com/erase-races-the-ameri...,"Tomball, TX",Thank you for taking the time to check out our...,"Erase Races, The American Unity Movement",dem,$0,"$150,000","Created February 23, 2019"


In [316]:
#will have to just drop the nulls for now
df=df.dropna()

In [317]:
df['created_date']=df.col_z.str.replace('Created','')

In [318]:
df.head(3)

,Unnamed: 0,funds,link,location,summary,title,tag,raised,goal,col_z,created_date
0,0,"\n$2,710 raised of $6,500",https://www.gofundme.com/steven-crowder-live-a...,"Dallas, TX",SMU College Republicans is bringing comedian S...,Steven Crowder Live at SMU!,rep,"$2,710","$6,500","Created March 7, 2018","March 7, 2018"
1,1,"\n$12,857 raised of $10,000",https://www.gofundme.com/reopen-a-nc-republica...,"Brookline, MA",NOTE: We reached our goal and then some in les...,Dems help reopen a NC Repub office,rep,"$12,857","$10,000","Created October 16, 2016","October 16, 2016"
3,3,"\n$3,649 raised of $5,000",https://www.gofundme.com/mugop,"Columbia, MO",We are the University of Missouri - College Re...,Mizzou College Republicans,rep,"$3,649","$5,000","Created November 17, 2015","November 17, 2015"


In [319]:
df=df.drop(columns=['funds','col_z'])

In [320]:
df.head(3)

,Unnamed: 0,link,location,summary,title,tag,raised,goal,created_date
0,0,https://www.gofundme.com/steven-crowder-live-a...,"Dallas, TX",SMU College Republicans is bringing comedian S...,Steven Crowder Live at SMU!,rep,"$2,710","$6,500","March 7, 2018"
1,1,https://www.gofundme.com/reopen-a-nc-republica...,"Brookline, MA",NOTE: We reached our goal and then some in les...,Dems help reopen a NC Repub office,rep,"$12,857","$10,000","October 16, 2016"
3,3,https://www.gofundme.com/mugop,"Columbia, MO",We are the University of Missouri - College Re...,Mizzou College Republicans,rep,"$3,649","$5,000","November 17, 2015"


In [329]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return float(x)

In [322]:
df['raised']=df.raised.str.replace('$','')
df['raised']=df.raised.str.replace(',','')
df['goal']=df.goal.str.replace('$','')
df['goal']=df.goal.str.replace(',','')

In [330]:
df['goal_int'] = df.goal.apply(value_to_float)
df['raised_int'] = df.raised.apply(value_to_float)


In [333]:
#put in csv for later
df.to_csv('merged_cleaned.csv',index=False)

# Looking at the Overlap
taking a look at which campaigns have the same links with a simple merge. we'll use some of the same cleaning


In [267]:
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)

In [291]:
intersect_df = pd.merge(dem_df,rep_df,on='link',how='inner',suffixes=('', '_y'))

In [292]:
#are there any fields that aren't the same?
# for index, row in intersect_df.iterrows():
#      # access data using column names
#     if row['title_x'] != row['title_y']:
#         print(row['title_x'],"isn't",row['title_y'] )
# #     print(index, row['funds_x'], row['funds_y'], row['title_x'])

In [334]:
intersect_df.head(10)

,Unnamed: 0,funds,link,location,summary,title,created_date,raised,goal
0,5,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-d...,"Lakehead, CA","Regardless of your political affiliation, you ...",Create The Democrat Dossier,"Created June 17, 2018",810,0.0
1,28,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",I have asked for a lot over the years! With th...,Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019",260,0.0
2,30,"\n$205 raised of $10,000",https://www.gofundme.com/help-progressive-demo...,"Lake George, MI",OUR STORY THANK YOU TO ALL THOSE WHO HAVE AL...,Help Progressive Democrats Win,NaN,205,0.0
3,42,"\n$100 raised of $225,000",https://www.gofundme.com/highergroundok,"Norman, OK",A new social fundraising movement to help get ...,Help us elect more Democrats in Oklahoma,"Created July 1, 2019",100,0.0
4,43,"\n$50 raised of $1,000",https://www.gofundme.com/a-democrat-for-amash,"Omaha, NE",Congressman Justin Amash stood up at a town ha...,A Democrat for Amash,"Created May 28, 2019",50,0.0
5,54,"\n$0 raised of $20,000",https://www.gofundme.com/democrat-parents-diso...,"Hillside, NJ",So basically i told my parents i am a republic...,Democrat Parents Disowns Republican,"Created November 6, 2018",0,0.0
6,64,"\n$0 raised of $150,000",https://www.gofundme.com/nmn67u-republican-or-...,"Fort Payne, AL",not gonna give a sob story I just need money. ...,family home and car,"Created June 11, 2019",0,0.0
7,65,"\n$0 raised of $11,000",https://www.gofundme.com/are-you-a-democrat-or...,"Philadelphia, PA",Republicans versus Democrats. We have -2- fam...,Are you a Democrat or a Republican,NaN,0,0.0
8,68,\n$0 raised of $1,https://www.gofundme.com/democrats-put-your-mo...,"Scarborough, ME",This is a political competition between Republ...,Democrats - Put Your Money Where Your Mouth Is!,"Created May 4, 2019",0,0.0
9,69,"\n$0 raised of $2,500",https://www.gofundme.com/inland-empire-young-d...,"Fontana, CA","Hi, I'm Jeremy Paul Ortiz, a proud citizen. I...",Inland Empire Young Democrats,"Created April 25, 2019",0,0.0


In [294]:
drop_y(intersect_df)

In [295]:
intersect_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 7 columns):
Unnamed: 0      284 non-null int64
funds           284 non-null object
link            284 non-null object
location        284 non-null object
summary         284 non-null object
title           284 non-null object
created_date    234 non-null object
dtypes: int64(1), object(6)
memory usage: 17.8+ KB


In [296]:
intersect_df=intersect_df[intersect_df['funds'].str.contains('\$')==True]
intersect_df['raised']=intersect_df.funds.str.extract(r"(.*) raised of")
intersect_df['goal']=intersect_df.funds.str.extract(r".* raised of (.*)")


In [297]:
intersect_df.head(5)

,Unnamed: 0,funds,link,location,summary,title,created_date,raised,goal
0,5,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-d...,"Lakehead, CA","Regardless of your political affiliation, you ...",Create The Democrat Dossier,"Created June 17, 2018",$810,"$75,000"
1,28,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",I have asked for a lot over the years! With th...,Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019",$260,$500
2,30,"\n$205 raised of $10,000",https://www.gofundme.com/help-progressive-demo...,"Lake George, MI",OUR STORY THANK YOU TO ALL THOSE WHO HAVE AL...,Help Progressive Democrats Win,NaN,$205,"$10,000"
3,42,"\n$100 raised of $225,000",https://www.gofundme.com/highergroundok,"Norman, OK",A new social fundraising movement to help get ...,Help us elect more Democrats in Oklahoma,"Created July 1, 2019",$100,"$225,000"
4,43,"\n$50 raised of $1,000",https://www.gofundme.com/a-democrat-for-amash,"Omaha, NE",Congressman Justin Amash stood up at a town ha...,A Democrat for Amash,"Created May 28, 2019",$50,"$1,000"


In [298]:
intersect_df['raised']=intersect_df.raised.str.replace('$','')
intersect_df['raised']=intersect_df.raised.str.replace(',','')
intersect_df['goal']=intersect_df.goal.str.replace('$','')
intersect_df['goal']=intersect_df.goal.str.replace(',','')

In [300]:
intersect_df.dropna()

,Unnamed: 0,funds,link,location,summary,title,created_date,raised,goal
0,5,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-d...,"Lakehead, CA","Regardless of your political affiliation, you ...",Create The Democrat Dossier,"Created June 17, 2018",810,75000
1,28,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",I have asked for a lot over the years! With th...,Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019",260,500
3,42,"\n$100 raised of $225,000",https://www.gofundme.com/highergroundok,"Norman, OK",A new social fundraising movement to help get ...,Help us elect more Democrats in Oklahoma,"Created July 1, 2019",100,225000
4,43,"\n$50 raised of $1,000",https://www.gofundme.com/a-democrat-for-amash,"Omaha, NE",Congressman Justin Amash stood up at a town ha...,A Democrat for Amash,"Created May 28, 2019",50,1000
5,54,"\n$0 raised of $20,000",https://www.gofundme.com/democrat-parents-diso...,"Hillside, NJ",So basically i told my parents i am a republic...,Democrat Parents Disowns Republican,"Created November 6, 2018",0,20000
6,64,"\n$0 raised of $150,000",https://www.gofundme.com/nmn67u-republican-or-...,"Fort Payne, AL",not gonna give a sob story I just need money. ...,family home and car,"Created June 11, 2019",0,150000
8,68,\n$0 raised of $1,https://www.gofundme.com/democrats-put-your-mo...,"Scarborough, ME",This is a political competition between Republ...,Democrats - Put Your Money Where Your Mouth Is!,"Created May 4, 2019",0,1
9,69,"\n$0 raised of $2,500",https://www.gofundme.com/inland-empire-young-d...,"Fontana, CA","Hi, I'm Jeremy Paul Ortiz, a proud citizen. I...",Inland Empire Young Democrats,"Created April 25, 2019",0,2500
10,72,"\n$0 raised of $50,000",https://www.gofundme.com/eqkyt-republican-or-d...,"Grand Junction, CO",Whose better?,Marvel vs. DC,"Created April 16, 2019",0,50000
12,99,\n$0 raised of $500,https://www.gofundme.com/6vey7-guns,"Orangevale, CA",I am on a mission to find out if I am a republ...,Democrat vs Republican,"Created November 20, 2018",0,500


,Unnamed: 0,funds,link,location,summary,title,created_date,raised,goal
0,5,"\n$810 raised of $75,000",https://www.gofundme.com/create-the-democrat-d...,"Lakehead, CA","Regardless of your political affiliation, you ...",Create The Democrat Dossier,"Created June 17, 2018",810,0.0
1,28,\n$260 raised of $500,https://www.gofundme.com/cosby2019,"Adamsville, AL",I have asked for a lot over the years! With th...,Cosby For Exec. VP of AL Young Democrats,"Created April 22, 2019",260,0.0
2,30,"\n$205 raised of $10,000",https://www.gofundme.com/help-progressive-demo...,"Lake George, MI",OUR STORY THANK YOU TO ALL THOSE WHO HAVE AL...,Help Progressive Democrats Win,NaN,205,0.0


# Cutting All Campaigns That Are Not in the USA
Seems like we can 1) get rid of rows with different currencies other than $ and 2) get rid of rows that don't have state abbreviations

In [1]:
#list of USA States:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
